In [60]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import sqlite3
import pandas as pd
import logging
import sys
from ingestion_db import ingest_db
import importlib
importlib.reload(logging)


sys.path.append('/content/drive/MyDrive/Colab Notebooks')


logging.basicConfig(
    filename='/content/drive/MyDrive/big_project/logs/get_vendor_summary.log',
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode='a'
)



def create_vendor_summary(conn):


  vendor_sales_summary = pd.read_sql_query("""
    WITH FreightSummary AS (
      SELECT VendorNumber, SUM(Freight) AS FreightCost
      FROM vendor_invoice
      GROUP BY VendorNumber
    ),
    PurchaseSummary AS (
      SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Volume,
        pp.Price AS ActualPrice,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
      FROM purchases p
      JOIN purchase_prices pp ON p.Brand = pp.Brand
      WHERE p.PurchasePrice > 0
      GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.PurchasePrice, pp.Volume, pp.Price
    ),
    SalesSummary AS (
      SELECT
        VendorNo,
        Brand,
        SUM(SalesDollars) AS TotalSalesDollars,
        AVG(SalesPrice) AS AvgSalesPrice,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(ExciseTax) AS TotalExciseTax
      FROM sales
      GROUP BY VendorNo, Brand
    )

    SELECT
      ps.VendorNumber,
      ps.VendorName,
      ps.Brand,
      ps.Description,
      ps.PurchasePrice,
      ps.ActualPrice,
      ps.Volume,
      ps.TotalPurchaseQuantity,
      ps.TotalPurchaseDollars,
      ss.TotalSalesQuantity,
      ss.TotalSalesDollars,
      ss.AvgSalesPrice,
      ss.TotalExciseTax,
      fs.FreightCost
    FROM PurchaseSummary ps
    LEFT JOIN SalesSummary ss
      ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
    LEFT JOIN FreightSummary fs
      ON ps.VendorNumber = fs.VendorNumber
    ORDER BY ps.TotalPurchaseDollars DESC
  """, conn)
  return vendor_sales_summary

def cleandata(df):
  df['Volume'] = df['Volume'].astype('float64')
  df.fillna(0, inplace= True)
  df['VendorName'] = df['VendorName'].str.strip()
  df['Description'] = df['Description'].str.strip()

  df['grossprofit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
  df['profitmargin'] =( df['grossprofit'] /  df['TotalSalesDollars']) * 100
  df['stockturnover'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
  df['salestopurchaseratio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']
  return df


















if __name__ == "__main__":

  conn = sqlite3.connect('/content/drive/MyDrive/big_project/inventory.db')

  logging.info('Creating vendor summary table.....')
  summary_df = create_vendor_summary(conn)
  logging.info(summary_df.head())

  logging.info('Cleaning data....')
  clean_df = cleandata(summary_df)
  logging.info(clean_df.head())

  logging.info('ingesting data....')
  ingest_db(clean_df , 'vendor_sales_summary',conn)
  logging.info('--completed--')